# テストの実行

In [1]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

In [2]:
TESTSET_DIR = os.path.join('/home', 'jupyter', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jupyter', 'work')

## マスタの読み込み

In [3]:
pre_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '01_test_pre.csv'), usecols=['seed'])
sys_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '02_test_sys.csv'), usecols=['seed'])
stress_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '03_test_stress.csv'), usecols=['seed'])

## 実行するロジックの指定

In [4]:
# 実行プログラムにタグをつけておく
PROG_TAG = 'bypass_avoid'

In [5]:
def solve(seed, N, M, D, K, problem, prog_path):
    start_time = time.perf_counter()
    
    command_str = 'echo {} | {}'.format(problem, prog_path)
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    #print('{}'.format(prob_id))    
    return (seed, N, M, D, K, e_time, res)

In [6]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')

    testset_path = os.path.join(TESTSET_DIR, testset_name+'.csv')
    testset_df = pd.read_csv(testset_path)

    with futures.ThreadPoolExecutor(max_workers=9) as executor:
        for _, row in testset_df.iterrows():
            seed, N, M, D, K, problem = row

            # バッチ実行
            future = executor.submit(solve, seed=seed,  N=N, M=M, D=D, K=K, problem=problem, prog_path=prog_path)
            future_list.append(future)

        _ = futures.as_completed(fs=future_list)

    for future in future_list:
        seed, N, M, D, K, e_time, res = future.result()

        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)
        solve_result.append(N)
        solve_result.append(M)
        solve_result.append(D)
        solve_result.append(K)

        # 経過時間
        solve_result.append(e_time)
        
        try:
            # コスト
            cost = int(res.stderr.decode('utf-8').split()[0].replace('Cost=', ''))

            # 非連結なノードペア数
            discon_cnt = int(res.stderr.decode('utf-8').split()[1].replace('DisconCnt=', ''))
                       
        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        solve_result.append(cost)
        solve_result.append(discon_cnt)

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    result_df.columns = ['testset', 'seed', 'N', 'M', 'D', 'K', 'time', 'cost', 'discon_cnt']

    return result_df

In [7]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [8]:
def get_summary_df(result_df):
    # 全体サマリ
    summary_all_df = pd.DataFrame()

    for testset in np.unique(result_df['testset']):
        test_result_df = result_df.query('testset == "{}"'.format(testset))

        summary_df = pd.DataFrame(
        {
            'testset': [testset],
            
            'time_mean': [int(np.mean(test_result_df['time']))],
            
            'cost_mean': [int(np.mean(test_result_df['cost']))],
            'cost_min': [min(test_result_df['cost'])],
            'cost_max': [max(test_result_df['cost'])],

            'discon_cnt_mean': [np.mean(test_result_df['discon_cnt'])],
            'discon_cnt_max': [max(test_result_df['discon_cnt'])],

            'time_max': [max(test_result_df['time'])],
        })

        summary_all_df = pd.concat([summary_all_df, summary_df], axis=0)   

    summary_all_df['tag'] = PROG_TAG
    summary_all_df = summary_all_df[['tag', 'testset', 'time_mean', 
                                     'cost_mean', 'cost_min', 'cost_max',
                                     'discon_cnt_mean', 'discon_cnt_max',
                                     'time_max']]
    
    return summary_all_df

In [15]:
PROG_NAME_LIST = ['main']
#PROG_NAME_LIST = ['main', 'main_off']

#TEST_LIST = ['00_test_mini']
testset_name = '01_test_pre'
#testset_name = '02_test_sys'
testset_name = '03_test_stress'

result_dict = {}
summary_all_dict = {}

for PROG_NAME in PROG_NAME_LIST:
    prog_path = os.path.join(PRJ_DIR, PROG_NAME)
    
    result_df = pd.DataFrame()
    
    testset_result_df = run_test(testset_name)
    result_df = pd.concat([result_df, testset_result_df], axis=0)
    
    result_dict[PROG_NAME] = result_df
    summary_all_dict[PROG_NAME] = get_summary_df(result_df)    

2023-01-29 05:44:00,084: Start
2023-01-29 06:14:56,356: finish!


In [20]:
# pre test
result_sub_df = pd.merge(pre_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '01_test_pre'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,time_max
0,max_min_euc_dist_bitset,01_test_pre,1867,55824340,3367131,639010524,336.84,2994,5830


In [21]:
# sys test
summary_df = pd.DataFrame()

if testset_name == '02_test_sys' or testset_name == '03_test_stress':
    result_sub_df = pd.merge(sys_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '02_test_sys'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,time_max
0,max_min_euc_dist_bitset,02_test_sys,1648,88770563,3072536,4242448330,405.338,18778,10607


In [23]:
# stress test
summary_df = pd.DataFrame()

if testset_name == '03_test_stress':
    result_sub_df = pd.merge(stress_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '03_test_stress'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,time_max
0,max_min_euc_dist_bitset,03_test_stress,1663,78819049,2935384,4242448330,376.2436,22128,10997


In [24]:
result_df.sort_values('cost')

,testset,seed,N,M,D,K,time,cost,discon_cnt
7917,03_test_stress,7917,890,2619,30,93,5089,2935384,0
7547,03_test_stress,7547,620,1825,30,63,1734,3021937,0
9511,03_test_stress,9511,936,2760,30,144,5230,3023338,0
2571,03_test_stress,2571,963,2830,30,119,5769,3063604,0
9402,03_test_stress,9402,973,2860,30,105,6056,3072536,0
...,...,...,...,...,...,...,...,...,...
1522,03_test_stress,1522,822,1440,5,341,1378,3505947820,11434
3670,03_test_stress,3670,934,1608,7,455,1476,3702022644,22128
9080,03_test_stress,9080,609,1057,5,251,329,4033042275,7246
427,03_test_stress,427,574,976,7,189,336,4037527619,9090


# 結果ログの保存

In [25]:
t_now = datetime.datetime.now() + datetime.timedelta(hours=9)
time_str = t_now.strftime('%Y%m%d_%H%M')

for PROG_NAME in PROG_NAME_LIST:
    result_df = result_dict[PROG_NAME]
    
    for testset in np.unique(result_df['testset']):
        csv_df = result_df.query('testset == "{}"'.format(testset))
        csv_df.to_csv(PRJ_DIR+'/result/{}_{}_{}_{}.csv'.format(time_str,PROG_TAG, testset, PROG_NAME), index=False)